In [2]:
import os

In [3]:
abs_path = '/home/Akshath/PycharmProjects/Summarize4Me'
os.chdir(abs_path)
%pwd

'/home/Akshath/PycharmProjects/Summarize4Me'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path


In [5]:
from Summarize4Me.constants import *
from Summarize4Me.utils.common import read_yaml, create_dirs

In [6]:
class ConfigManager:
    def __init__(
        self,
        config_fpath = CONFIG_FILE_PATH,
        params_fpath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_fpath)
        self.params = read_yaml(params_fpath)

        create_dirs([self.config.artifacts_root])

    def get_model_eval_cfg(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_dirs([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )

        return model_evaluation_config

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas
from tqdm import tqdm

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_chunks(self, list_elements, batch_size):
        for i in range(0, len(list_elements), batch_size):
            yield list_elements[i: i + batch_size]

    def calculate_metric(self, dataset, metric, model, tokenizer,
                         batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                         column_text="article",
                         column_summary="highlights"):

        article_batches = list(self.generate_batch_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=2048, truncation=True, padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device), length_penalty=0.8, num_beams=8, max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_space=True) for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = load_metric('rouge')

        score = self.calculate_metric(
            dataset_samsum_pt['test'][0:10], rouge_metric, pegasus, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pandas.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)




In [9]:
try:
    config = ConfigManager()
    model_eval_config = config.get_model_eval_cfg()
    model_eval_config = ModelEvaluation(config = model_eval_config)
    model_eval_config.evaluate()
except Exception as e:
    raise e

[2023-06-11 01:05:20,064: INFO: common: YAML File: config/config.yaml loaded]
[2023-06-11 01:05:20,068: INFO: common: YAML File: params.yaml loaded]
[2023-06-11 01:05:20,069: INFO: common: Created dir at: artifacts]
[2023-06-11 01:05:20,070: INFO: common: Created dir at: artifacts/model_evaluation]


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'artifacts/model_trainer/tokenizer'. Use `repo_type` argument if needed.